# Magnitude scale calibration

In [ ]:
import pandas as pd
import numpy as np
import calibration
import pickle
import matplotlib.pyplot as plt

## Load the feature matrix
Required columns in the feature matrix are:
- the feature to generate the magnitude from
- distance (DIST) and depth (DEPTH) in km
- latitude (LAT) and longitude (LON)
- reference magnitude (M_EXT)
- station (STATION)
- a unique event identifier (EVENT)

In [ ]:
features = pd.read_csv('IPOC_features.csv')

## Train-Dev-Test split

In [ ]:
features_train = features[features['SPLIT'] == 'TRAIN']
features_dev = features[features['SPLIT'] == 'DEV']
features_test = features[features['SPLIT'] == 'TEST']

## Calibrate a model from data
*Warning:* This step has a runtime of multiple hour and requires >150 GB of memory. Disable the knn correction for significantly lower runtime and memory consumption. Be aware that this changes the function's signature and return values.

In [ ]:
metric = 'DISP_NE'   # Name of the column with the feature
x_ref = (np.linspace(0, 500, 50),   # Distance spacing
         np.linspace(0, 200, 20))   # Depth spacing

s, g, nn = calibration.estimate_attenuation_correction(data=features_train,
                                                       metric=metric,
                                                       x_ref=x_ref,
                                                       knn_correction=True)

## Alternative: Load a precalibrated model

In [ ]:
s, g, nn, x_ref = pickle.load(open(f'models/{metric}.pkl', 'rb'))

## Determine magnitude values
Writes single station magnitude estimates into the column PRED_{metric}

In [ ]:
calibration.add_prediction(data=features,
                           metric=metric,
                           s=s, g=g, nn=nn, x_ref=x_ref)

## Calculate event means and residuals
Writes event means and station residuals into the columns MEAN_PRED_{metric} and RESIDUAL_PRED_{metric}

In [ ]:
calibration.calc_means(data=features,
                       metric=metric)

## Analyse RMSE

In [ ]:
rmse_train = np.sqrt(np.nanmean(features_train[f'RESIDUAL_PRED_{metric}']**2))
rmse_dev = np.sqrt(np.nanmean(features_dev[f'RESIDUAL_PRED_{metric}']**2))
rmse_test = np.sqrt(np.nanmean(features_test[f'RESIDUAL_PRED_{metric}']**2))

print(f'RMSE train: {rmse_train:.3f}\nRMSE dev: {rmse_dev:.3f}\nRMSE test: {rmse_test:.3f}')

## Visualize attenuation

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)
cb = calibration.distance_depth_correction(x_ref=x_ref, g=g, ax=ax)
fig.colorbar(cb)

## Remove outliers
In addition recalculate means and residuals

In [ ]:
features = calibration.remove_outliers(data=features, metric=f'PRED_{metric}')
calibration.calc_means(data=features, metric=metric)

## Train boosting trees
Trains three overlapping boosting trees to get non-overfitted magnitude estimates for each measurment. The final values are available in the columns PRED_BOOST_{metric} for station estimates and MEAN_PRED_BOOST_{metric} for network estimates.

In [ ]:
data = data[~np.isnan(data[f'MEAN_PRED_{metric}'])]
features = calibration.create_boosting_scale(data=data, metric=metric)